In [ ]:
# hide
# skip
!git clone https://github.com/benihime91/gale # install gale on colab
!pip install -e "gale[dev]"

In [ ]:
# default_exp classification.modelling.meta_arch.common

In [ ]:
# hide
%load_ext nb_black
%load_ext autoreload
%autoreload 2
%matplotlib inline

<IPython.core.display.Javascript object>

In [ ]:
# hide
import warnings

from nbdev.export import *
from nbdev.showdoc import *

warnings.filterwarnings("ignore")

<IPython.core.display.Javascript object>

# Meta Architectures : Generalized Image Classifier 
> Default Model Architecture for Image Classification

In [ ]:
# export
import logging
from typing import *

import torch
from omegaconf import DictConfig, OmegaConf
from pytorch_lightning.core.memory import get_human_readable_count
from torch.nn import Module

from gale.classification.modelling.backbones import ImageClassificationBackbone
from gale.classification.modelling.build import build_backbone, build_head
from gale.classification.modelling.heads import ImageClassificationHead
from gale.core.classes import GaleModule
from gale.core.nn.shape_spec import ShapeSpec

_logger = logging.getLogger(__name__)

<IPython.core.display.Javascript object>

In [ ]:
# export
class GeneralizedImageClassifier(GaleModule):
    """
    A General Image Classifier. Any models that contains the following 2 components:
    1. Feature extractor (aka backbone)
    2. Image Classification head (Pooling + Classifier)
    """

    def __init__(
        self,
        backbone: ImageClassificationBackbone,
        head: ImageClassificationHead,
    ):
        """
        Arguments:
        1. `backbone`: a `ImageClassificationBackbone` module, must follow gale's backbone interface
        2. `head`: a head containg the classifier. and the pooling layer, must be an instance of
        `ImageClassificationHead`.
        """
        super(GeneralizedImageClassifier, self).__init__()
        self.backbone = backbone
        assert isinstance(backbone, ImageClassificationBackbone)
        self.head = head
        assert isinstance(head, ImageClassificationHead)

    def forward(self, batched_inputs: torch.Tensor) -> torch.Tensor:
        """
        Runs the batched_inputs through `backbone` followed by the `head`.
        Returns a Tensor which contains the logits for the batched_inputs.
        """
        # forward pass through the backbone
        out = self.backbone(batched_inputs)
        # pass through the classification layer
        out = self.head(out)
        return out

    @classmethod
    def from_config_dict(cls, cfg: DictConfig):
        """
        Instantiate the Meta Architecture from gale config
        """
        if not hasattr(cfg.model, "backbone"):
            raise ValueError("Configuration for model backbone not found")

        if not hasattr(cfg.model, "head"):
            raise ValueError("Configuration for model head not found")

        input_shape = ShapeSpec(cfg.input.channels, cfg.input.height, cfg.input.width)
        _logger.debug(f"Inputs: {input_shape}")

        backbone = build_backbone(cfg, input_shape=input_shape)
        param_count = get_human_readable_count(
            sum([m.numel() for m in backbone.parameters()])
        )
        _logger.debug(
            "Backbone {} created, param count: {}.".format(
                cfg.model.backbone.name, param_count
            )
        )

        head = build_head(cfg, backbone.output_shape())
        param_count = get_human_readable_count(
            sum([m.numel() for m in head.parameters()])
        )
        _logger.debug(
            "Head {} created, param count: {}.".format(cfg.model.head.name, param_count)
        )

        kwds = {"backbone": backbone, "head": head}

        instance = cls(**kwds)
        instance.input_shape = input_shape

        param_count = get_human_readable_count(
            sum([m.numel() for m in instance.parameters()])
        )
        _logger.info("Model created, param count: {}.".format(param_count))

        return instance

    def build_param_dicts(self):
        """
        Builds up the Paramters dicts for optimization
        """
        backbone_params = self.backbone.build_param_dicts()
        head_params = self.head.build_param_dicts()
        parameters = backbone_params + head_params

        # filter and remove any empty paramter groups if any
        pgs_filterd = []

        for group in parameters:
            if group["params"] == []:
                pass
            else:
                pgs_filterd += [group]
        return pgs_filterd

    def get_lrs(self) -> List:
        """
        Returns a List containining the Lrs' for
        each parameter group. This is required to build schedulers
        like `torch.optim.lr_scheduler.OneCycleScheduler` which needs
        the max lrs' for all the Param Groups.
        """
        lrs = []
        for p in self.build_param_dicts():
            lrs.append(p["lr"])
        return lrs

<IPython.core.display.Javascript object>

This model architecture will work for most common computer vision fietuning use case. We take a `backbone` and `classifier`. We run the input through the backbone to extract the feature_maps which are then used by the classifier to given predictions on the Input. The paramters dicts for optimization are generated by the `backbone` and the `head` itself.

> Note: For advanced use cases you might want to create a model. A model muse inherit from `GaleModule` and be registered in `META_ARCH_REGISTRY`. Your model should also have the following methods to work in the Gale ecosystem.

In [ ]:
show_doc(GeneralizedImageClassifier.from_config_dict)

<h4 id="GeneralizedImageClassifier.from_config_dict" class="doc_header"><code>GeneralizedImageClassifier.from_config_dict</code><a href="__main__.py#L37" class="source_link" style="float:right">[source]</a></h4>

> <code>GeneralizedImageClassifier.from_config_dict</code>(**`cfg`**:`DictConfig`)

Instantiate the Meta Architecture from gale config

<IPython.core.display.Javascript object>

In [ ]:
show_doc(GeneralizedImageClassifier.forward)

<h4 id="GeneralizedImageClassifier.forward" class="doc_header"><code>GeneralizedImageClassifier.forward</code><a href="__main__.py#L26" class="source_link" style="float:right">[source]</a></h4>

> <code>GeneralizedImageClassifier.forward</code>(**`batched_inputs`**:`Tensor`)

Runs the batched_inputs through `backbone` followed by the `head`.
Returns a Tensor which contains the logits for the batched_inputs.

<IPython.core.display.Javascript object>

In [ ]:
show_doc(GeneralizedImageClassifier.build_param_dicts)

<h4 id="GeneralizedImageClassifier.build_param_dicts" class="doc_header"><code>GeneralizedImageClassifier.build_param_dicts</code><a href="__main__.py#L81" class="source_link" style="float:right">[source]</a></h4>

> <code>GeneralizedImageClassifier.build_param_dicts</code>()

Builds up the Paramters dicts for optimization

<IPython.core.display.Javascript object>

`Meta_Arch`'s can also be instatiated via a appropriate config file. Let's see how ..

In [ ]:
from dataclasses import dataclass, field
from omegaconf import MISSING, OmegaConf
from gale.classification.modelling.backbones import ResNetBackbone
from gale.classification.modelling.heads import FastaiHead

<IPython.core.display.Javascript object>

For a meta_arch we first need to create the configurations for the `Backbone` and the `Head` of the model. These 
must be registerd in `IMAGE_CLASSIFICATION_BACKBONES` and `IMAGE_CLASSIFICATION_HEADS` Registy respectively. The instances are automatically instiated by the `GeneralizedImageClassifier` meta_arch.

In [ ]:
@dataclass
class C_Backbone:
    model_name: str = "resnet18"
    pretrained: bool = True
    act: Any = None
    lr: Any = 1e-05
    wd: Any = 1e-05
    freeze_at: int = 4


@dataclass
class C_Head:
    num_classes: int = 2
    drop_rate: Any = 0.3
    lr: Any = 2e-03
    wd: Any = 1e-05
    filter_wd: Any = False

<IPython.core.display.Javascript object>

We also need a few more things and the config must be composed in a gale config style. We need the definitions of the input like channels, height and weight. So let's compose these -

In [ ]:
b_args = OmegaConf.structured(C_Backbone())
h_args = OmegaConf.structured(C_Head())

# Backbone config
b = OmegaConf.create()
b.name = "ResNetBackbone"
b.init_args = b_args

# Head config
h = OmegaConf.create()
h.name = "FullyConnectedHead"
h.init_args = h_args

i = OmegaConf.create()
i.channels = 3
i.height = 224
i.width = 224

m = OmegaConf.create()
m.backbone = b
m.head = h

<IPython.core.display.Javascript object>

In [ ]:
conf = OmegaConf.create(dict(input=i, model=m))
# print(OmegaConf.to_yaml(conf))

<IPython.core.display.Javascript object>

In [ ]:
m = GeneralizedImageClassifier.from_config_dict(conf)
shape = (m.input_shape.channels, m.input_shape.height, m.input_shape.width)
inp = torch.randn(2, *shape)
o = m(inp)

<IPython.core.display.Javascript object>

In [ ]:
# hide
import pytorch_lightning as pl
import torchmetrics
import torchvision.transforms as T
from fastcore.all import Path
from nbdev.export import Config
from torch import optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

from gale.collections.callbacks import NotebookProgressCallback
from gale.collections.download import download_and_extract_archive
from gale.core.nn.optim.lr_schedulers import WarmupStepLR
from gale.core.utils.visualize import show_images

URL = "https://download.pytorch.org/tutorial/hymenoptera_data.zip"
data_path = Path(Config().path("nbs_path")) / "data"

# download a toy dataset
download_and_extract_archive(url=URL, download_root=data_path)

Using downloaded and verified file: /Users/ayushman/Desktop/gale/nbs/data/hymenoptera_data.zip
Extracting /Users/ayushman/Desktop/gale/nbs/data/hymenoptera_data.zip to /Users/ayushman/Desktop/gale/nbs/data


<IPython.core.display.Javascript object>

In [ ]:
# hide
# fmt:off
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': T.Compose([
        T.RandomResizedCrop(224),
        T.RandomHorizontalFlip(),
        T.ToTensor(),
        T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': T.Compose([
        T.Resize(256),
        T.CenterCrop(224),
        T.ToTensor(),
        T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

training_data = ImageFolder(data_path / "hymenoptera_data/train", transform=data_transforms["train"])
validation_data = ImageFolder(data_path / "hymenoptera_data/val", transform=data_transforms["val"])

train_dl = DataLoader(training_data, batch_size=32, shuffle=True)
valid_dl = DataLoader(validation_data, batch_size=32, shuffle=False)

<IPython.core.display.Javascript object>

In [ ]:
# hide
# fmt:off
class Learner(pl.LightningModule):
    def __init__(self, model: GeneralizedImageClassifier):
        super().__init__()
        self.model = model
        self.train_metric = torchmetrics.Accuracy()
        self.valid_metric = torchmetrics.Accuracy()
        self.loss_fn = torch.nn.CrossEntropyLoss()

    def forward(self, xb):
        return self.model(xb)

    def training_step(self, batch: Any, batch_idx: int):
        x, y = batch
        y_hat = self(x)
        loss = self.loss_fn(y_hat, y)
        acc = self.train_metric(torch.nn.functional.softmax(y_hat), y)
        self.log_dict(dict(loss=loss, acc=acc))
        return loss

    def validation_step(self, batch: Any, batch_idx: int):
        x, y = batch
        y_hat = self(x)
        loss = self.loss_fn(y_hat, y)
        acc = self.valid_metric(torch.nn.functional.softmax(y_hat), y)
        self.log_dict(dict(val_loss=loss, val_acc=acc))

    def configure_optimizers(self):
        paramters = self.model.build_param_dicts()
        opt = optim.AdamW(paramters)
        sch = WarmupStepLR(opt, num_decays=3, warmup_epochs=3, decay_rate=0.1, epochs=self.trainer.max_epochs)
        return [opt], [sch]

<IPython.core.display.Javascript object>

In [ ]:
# hide
# fmt:off
# slow
cbs = [
    NotebookProgressCallback(), 
    pl.callbacks.LearningRateMonitor(logging_interval="step", log_momentum=True)
]

logger = pl.loggers.TensorBoardLogger(save_dir="lightning_logs/", name="my_model", 
                                      log_graph=True, default_hp_metric=False)

trainer = pl.Trainer(max_epochs=15, callbacks=cbs, log_every_n_steps=1, logger=logger)
model = GeneralizedImageClassifier.from_config_dict(conf)
learn = Learner(model)

trainer.fit(learn, train_dataloader=train_dl, val_dataloaders=valid_dl)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
Missing logger folder: lightning_logs/my_model

  | Name         | Type                       | Params
------------------------------------------------------------
0 | model        | GeneralizedImageClassifier | 11.2 M
1 | train_metric | Accuracy                   | 0     
2 | valid_metric | Accuracy                   | 0     
3 | loss_fn      | CrossEntropyLoss           | 0     
------------------------------------------------------------
8.4 M     Trainable params
2.8 M     Non-trainable params
11.2 M    Total params
44.710    Total estimated model params size (MB)


epoch,val_loss,val_acc,loss,acc,time,samples/s
0,0.820101,0.490196,1.074716,0.450000,31.792200,0.408900
1,0.683885,0.542484,0.769564,0.400000,31.616300,0.411200
2,0.455971,0.810458,0.380922,0.900000,32.251600,0.403100
3,0.267772,0.928105,0.317305,0.850000,32.415400,0.401000
4,0.264430,0.941176,0.334841,0.850000,32.588500,0.398900
5,0.259231,0.941176,0.260302,0.900000,32.836700,0.395900
6,0.256384,0.941176,0.310489,0.850000,33.339500,0.389900
7,0.251130,0.941176,0.432859,0.800000,30.974400,0.419700
8,0.253889,0.934641,0.270440,0.950000,32.056600,0.405500
9,0.252289,0.941176,0.409051,0.800000,31.858300,0.408100


1

<IPython.core.display.Javascript object>

## Export-

In [ ]:
# hide
notebook2script()

Converted 00_core.utils.logger.ipynb.
Converted 00a_core.utils.visualize.ipynb.
Converted 00b_core.utils.structures.ipynb.
Converted 01_core.nn.utils.ipynb.
Converted 01a_core.nn.losses.ipynb.
Converted 02_core.nn.optim.optimizers.ipynb.
Converted 02a_core.nn.optim.lr_schedulers.ipynb.
Converted 03_core.classes.ipynb.
Converted 04_classification.modelling.backbones.ipynb.
Converted 04a_classification.modelling.heads.ipynb.
Converted 04b_classification.modelling.meta_arch.common.ipynb.
Converted 04b_classification.modelling.meta_arch.vit.ipynb.
Converted 05_classification.data.common.ipynb.
Converted 05a_classification.data.transforms.ipynb.
Converted 05b_classification.data.build.ipynb.
Converted 06_classification.task.ipynb.
Converted 07_collections.pandas.ipynb.
Converted 07a_collections.callbacks.notebook.ipynb.
Converted 07b_collections.callbacks.ema.ipynb.
Converted index.ipynb.


<IPython.core.display.Javascript object>